In [4]:
previous_model = True

In [19]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
import os.path as path
import os
os.environ['OMP_NUM_THREADS'] = '2'

import lib.dist as dist
import lib.utils as utils


from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update({
    'font.family' : 'STIXGeneral',
    'mathtext.fontset' : 'stix', 
    'xtick.labelsize' : 13,
    'xtick.top' : True,
    'xtick.direction' : 'in',
    'ytick.direction' : 'in',
    'ytick.labelsize' : 13, 
    'ytick.right' : True, 
    'axes.labelsize' : 16,
    'legend.frameon': False,
    'legend.fontsize': 13,
    'legend.handlelength' : 1.5,
    'savefig.dpi' : 600,
    'savefig.bbox' : 'tight'
})
torch.__version__

'1.7.1'

In [4]:
b = 60

latent_dim = 8
beta = 4
prior_dist = dist.Normal()
q_dist = dist.Normal()

txt = np.loadtxt(f'../1_QMC/train/Bethe_14_beta{b:d}/field-2.10.dat')
s_num, s_len = txt.shape
print("txt.shape =", s_num, s_len)
s_train = 1000
s_test = s_num-s_train
if (s_train+s_test > s_num): print("WARNING : not enough datapoints")



txt.shape = 3000 288


In [ ]:

U = np.array([0.01*i for i in range(100, 401)])
U_up, U_dn = [], []
for i, u in enumerate(U):
    if path.isfile(f'../1_QMC/train/Bethe_14_beta{b:d}/field-{u:.2f}.dat')==True: U_up.append(u)
    if path.isfile(f'../1_QMC/train/Bethe_41_beta{b:d}/field-{u:.2f}.dat')==True: U_dn.append(u)
U_up = np.array(U_up)
U_dn = np.array(U_dn)

#spin2d_up_train = np.zeros((len(U_up)*s_train, 1, 17, 17))
#spin2d_dn_train = np.zeros((len(U_dn)*s_train, 1, 17, 17))
spin2d_up_test = np.zeros((len(U_up)*s_test, 1, 17, 17))
spin2d_dn_test = np.zeros((len(U_dn)*s_test, 1, 17, 17))
print("U_up, U_dn len =", len(U_up), len(U_dn))
#print("spin2d_up_train size =", spin2d_up_train.shape)
#print("spin2d_dn_train size =", spin2d_dn_train.shape)

txt = np.zeros((s_num, s_len+1))
for i, u in enumerate(U_up):
    txt[:,:-1] = np.loadtxt(f'../1_QMC/train/Bethe_14_beta{b:d}/field-{u:.2f}.dat')
    txt = 0.5*txt+0.5
    #spin2d_up_train[i*s_train:(i+1)*s_train] = txt[:s_train,:].reshape(s_train, 1, 17, 17)
    spin2d_up_test[i*s_test:(i+1)*s_test] = txt[-s_test:,:].reshape(s_test, 1, 17, 17)
    spin2d_up[i*s_num:(i+1)*s_num] = txt.reshape(s_num, 1, 17, 17)
for i, u in enumerate(U_dn):
    txt[:,:-1] = np.loadtxt(f'../1_QMC/train/Bethe_41_beta{b:d}/field-{u:.2f}.dat')
    txt = 0.5*txt+0.5
    #spin2d_dn_train[i*s_train:(i+1)*s_train] = txt[:s_train,:].reshape(s_train, 1, 17, 17)
    spin2d_dn_test[i*s_test:(i+1)*s_test] = txt[-s_test:,:].reshape(s_test, 1, 17, 17)
    spin2d_dn[i*s_num:(i+1)*s_num] = txt.reshape(s_num, 1, 17, 17)


spin2d = np.concatenate([spin2d_up, spin2d_dn])
spin2d = torch.Tensor(spin2d)
print("spin2d size =", spin2d.size())

In [ ]:
class CEncoder(nn.Module):
    def __init__(self, output_dim):
        super(CEncoder, self).__init__()
        self.output_dim = output_dim

        #self.conv1 = nn.Conv2d(1, 16, 4, 2, 1)  # 16
        #self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(1, 16, 5, 2, 1)  # 8
        self.bn2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 32, 4, 2, 1)  # 4
        self.bn3 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(32, 256, 4)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv_z = nn.Conv2d(256, output_dim, 1)

        # setup the non-linearity
        self.act = nn.SELU()

    def forward(self, x):
        h = x.view(-1, 1, 17, 17)
        #h = self.act(self.bn1(self.conv1(h)))
        h = self.act(self.bn2(self.conv2(h)))
        h = self.act(self.bn3(self.conv3(h)))
        h = self.act(self.bn4(self.conv4(h)))
        z = self.conv_z(h).view(x.size(0), self.output_dim)
        return z
    
class CDecoder(nn.Module):
    def __init__(self, input_dim):
        super(CDecoder, self).__init__()
        self.conv1 = nn.ConvTranspose2d(input_dim, 256, 1, 1, 0)  # 1 x 1
        self.bn1 = nn.BatchNorm2d(256)
        self.conv2 = nn.ConvTranspose2d(256, 32, 4, 1, 0)  # 4
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.ConvTranspose2d(32, 16, 4, 2, 1)  # 8
        self.bn3 = nn.BatchNorm2d(16)
        #self.conv4 = nn.ConvTranspose2d(16, 16, 4, 2, 1)  # 16
        #self.bn4 = nn.BatchNorm2d(16)
        self.conv_final = nn.ConvTranspose2d(16, 1, 5, 2, 1) #17

        # setup the non-linearity
        self.act = nn.SELU()

    def forward(self, z):
        h = z.view(z.size(0), z.size(1), 1, 1)
        h = self.act(self.bn1(self.conv1(h)))
        h = self.act(self.bn2(self.conv2(h)))
        h = self.act(self.bn3(self.conv3(h)))
        #h = self.act(self.bn4(self.conv4(h)))
        mu_img = self.conv_final(h)
        return mu_img
    
def logsumexp(value, dim=None, keepdim=False):
    """Numerically stable implementation of the operation
    value.exp().sum(dim, keepdim).log()
    """
    if dim is not None:
        m, _ = torch.max(value, dim=dim, keepdim=True)
        value0 = value - m
        if keepdim is False:
            m = m.squeeze(dim)
        return m + torch.log(torch.sum(torch.exp(value0),
                                       dim=dim, keepdim=keepdim))
    else:
        m = torch.max(value)
        sum_exp = torch.sum(torch.exp(value - m))
        if isinstance(sum_exp, Number):
            return m + math.log(sum_exp)
        else:
            return m + torch.log(sum_exp)
        
def display_samples(model, x):
    fig, ax = plt.subplots(1,2, figsize = (15, 3))
    th = 1
    sample_mu = model.model_sample(batch_size=100).sigmoid()
    xs, x_params, zs, z_params = model.reconstruct_img(x)
    ax[0].imshow(x[th,0], aspect='auto')
    ax[1].imshow(xs[th,0], aspect='auto')
    #ax[2].imshow(x_params[th,0].detach().numpy())
    #ax[3].imshow(z_params[:,:,0].detach().numpy())
    #ax[4].imshow(z_params[:,:,1].detach().numpy())
    plt.show()

In [ ]:
### beta-TCVAE
class TCVAE(nn.Module):
    def __init__(self, z_dim, beta, prior_dist=dist.Normal(), q_dist=dist.Normal(),
                 include_mutinfo=True, tcvae=True, mss=True):
        super(TCVAE, self).__init__()
        
        self.z_dim = z_dim
        self.include_mutinfo = include_mutinfo
        self.tcvae = tcvae
        self.lamb = 0
        self.beta = beta
        self.mss = mss
        self.x_dist = dist.Bernoulli()

        self.prior_dist = prior_dist
        self.q_dist = q_dist
        self.register_buffer('prior_params', torch.zeros(self.z_dim, 2))

        self.encoder = CEncoder(z_dim * self.q_dist.nparams)
        self.decoder = CDecoder(z_dim)

    def _get_prior_params(self, batch_size=1):
        expanded_size = (batch_size,) + self.prior_params.size()
        prior_params = Variable(self.prior_params.expand(expanded_size))
        return prior_params
    
    def model_sample(self, batch_size=1):
        prior_params = self._get_prior_params(batch_size)
        zs = self.prior_dist.sample(params=prior_params)
        x_params = self.decoder.forward(zs)
        return x_params

    def encode(self, x):
        x = x.view(x.size(0), 1, 17, 17)
        z_params = self.encoder.forward(x).view(x.size(0), self.z_dim, self.q_dist.nparams)
        zs = self.q_dist.sample(params=z_params)
        return zs, z_params

    def decode(self, z):
        x_params = self.decoder.forward(z).view(z.size(0), 1, 17, 17)
        xs = self.x_dist.sample(params=x_params)
        return xs, x_params

    def reconstruct_img(self, x):
        zs, z_params = self.encode(x)
        xs, x_params = self.decode(zs)
        return xs, x_params, zs, z_params
    
    def _log_importance_weight_matrix(self, batch_size, dataset_size):
        N = dataset_size
        M = batch_size - 1
        strat_weight = (N - M) / (N * M)
        W = torch.Tensor(batch_size, batch_size).fill_(1 / M)
        W.view(-1)[::M+1] = 1 / N
        W.view(-1)[1::M+1] = strat_weight
        W[M-1, 0] = strat_weight
        return W.log()
    
    def elbo(self, x, dataset_size):
        batch_size = x.size(0) 
        x = x.view(batch_size, 1, 17, 17)
        prior_params = self._get_prior_params(batch_size)
        x_recon, x_params, zs, z_params = self.reconstruct_img(x)
        logpx = self.x_dist.log_density(x, params=x_params).view(batch_size, -1).sum(1)
        logpz = self.prior_dist.log_density(zs, params=prior_params).view(batch_size, -1).sum(1)
        logqz_condx = self.q_dist.log_density(zs, params=z_params).view(batch_size, -1).sum(1) 

        elbo = logpx + logpz - logqz_condx # log p(x|z) + log p(z) - log q(z|x)
        # log q(z) ~ log 1/(NM) sum_m=1^M q(z|x_m) = - log(MN) + logsumexp_m(q(z|x_m))
        _logqz = self.q_dist.log_density(zs.view(batch_size, 1, self.z_dim),
                                         z_params.view(1, batch_size, self.z_dim, self.q_dist.nparams))

        if not self.mss: # minibatch weighted sampling
            logqz_prodmarginals = (logsumexp(_logqz, dim=1, keepdim=False) - math.log(batch_size * dataset_size)).sum(1)
            logqz = (logsumexp(_logqz.sum(2), dim=1, keepdim=False) - math.log(batch_size * dataset_size))
        else: # minibatch stratified sampling
            logiw_matrix = Variable(self._log_importance_weight_matrix(batch_size, dataset_size).type_as(_logqz.data))
            logqz = logsumexp(logiw_matrix + _logqz.sum(2), dim=1, keepdim=False)
            logqz_prodmarginals = logsumexp(
                logiw_matrix.view(batch_size, batch_size, 1) + _logqz, dim=1, keepdim=False).sum(1)

        if self.include_mutinfo:
            elbo2 = logpx - logqz_condx + logqz - self.beta * (logqz - logqz_prodmarginals) - \
                                (1 - self.lamb) * (logqz_prodmarginals - logpz)
        else:
            elbo2 = logpx - self.beta * (logqz - logqz_prodmarginals) - \
                                (1 - self.lamb) * (logqz_prodmarginals - logpz)

        return elbo2, elbo.detach()


In [ ]:
if previous_model == True:
    model = TCVAE(z_dim=latent_dim, beta=beta, prior_dist=prior_dist, q_dist=q_dist)
    model.load_state_dict(torch.load('./frozen/VAE_CNN_2.pth'))

In [ ]:
if previous_model == False:
    model = TCVAE(z_dim=latent_dim, beta=beta, prior_dist=prior_dist, q_dist=q_dist)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    train_loader=DataLoader(dataset=spin2d, batch_size=1000, shuffle=True)

    train_elbo = []
    dataset_size = len(train_loader.dataset)
    num_iter = len(train_loader)*100
    print("dataset_size, num_iter =", dataset_size, num_iter)
    elbo_run_mean = utils.RunningAverageMeter()

    for n in range(num_iter):
        for i,x in enumerate(train_loader):
            model.train()
            optimizer.zero_grad()
            obj, elbo = model.elbo(x, dataset_size)
            if utils.isnan(obj).any(): break
            obj.mean().mul(-1).backward()
            elbo_run_mean.update(elbo.mean().item())
            optimizer.step()

        if n%50==0:
            train_elbo.append(elbo_run_mean.avg)
            print('[n = %03d] ELBO = %.6f (%.6f)'%(n, elbo_run_mean.val, elbo_run_mean.avg))
            display_samples(model, x)

In [ ]:
spin2dtest = np.concatenate([spin2d_up_test, spin2d_dn_test])
spin2dtest = torch.Tensor(spin2dtest)
xs, x_params, zs, z_params = model.reconstruct_img(spin2dtest)

test_mu = z_params[:, :, 0].detach().numpy()
test_logsig = z_params[:, :, 1].detach().numpy()
test_var = np.exp(test_logsig * 2)
fig, ax = plt.subplots(1,2, figsize = (6, 6))
ax[0].imshow(test_mu, aspect = 'auto')
ax[1].imshow(test_var, aspect = 'auto')

In [ ]:
U_len = len(U_up)+len(U_dn)
spin2d_up_test = torch.Tensor(spin2d_up_test)
spin2d_dn_test = torch.Tensor(spin2d_dn_test)
#xs, x_params, zs, z_params = model.reconstruct_img(spin2dtest)
#xs_up, x_params_up, zs_up, z_params_up = model.reconstruct_img(spin2d_up_test)
#xs_dn, x_params_dn, zs_dn, z_params_dn = model.reconstruct_img(spin2d_dn_test)

plt.imshow(spin2dtest.reshape(U_len*s_test, 289), aspect='auto')
cd = plt.colorbar()
plt.show()

plt.imshow(xs.reshape(U_len*s_test, 289), aspect='auto')
cd = plt.colorbar()
plt.show()

In [ ]:
test_ens_mu = np.zeros((134, latent_dim))
test_ens_logsig = np.zeros((134, latent_dim))
for i in range(134):
    test_ens_mu[i] = np.average(z_params[i*s_test:(i+1)*s_test, :, 0].detach().numpy(), axis=0)
    test_ens_logsig[i] = np.average(z_params[i*s_test:(i+1)*s_test, :, 1].detach().numpy(), axis=0)
test_ens_var = np.exp(test_ens_logsig * 2)
fig, ax = plt.subplots(1,2, figsize = (6, 6))
ax[0].imshow(test_ens_mu, aspect = 'auto')
ax[1].imshow(test_ens_var, aspect = 'auto')

In [ ]:
if previous_model == False: torch.save(model.state_dict(), 'frozen/VAE_CNN_1.pth')

# PCA

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
#scaler1 = StandardScaler()
#scaler1.fit(test_mu)
#mu_scaled = scaler1.transform(test_mu)
pca1 = PCA(n_components=3)
pca1.fit(test_ens_mu)
mu_pca = pca1.transform(test_ens_mu)

#scaler2 = StandardScaler()
#scaler2.fit(test_var)
#var_scaled = scaler2.transform(test_var)
pca2 = PCA(n_components=2)
pca2.fit(test_ens_var)
var_pca = pca2.transform(test_ens_var)

UU = np.concatenate([U_up, U_dn])

plt.plot(UU, mu_pca[:,0], 'b.-')
plt.plot(UU, mu_pca[:,1], 'r.-')
plt.show()

plt.plot(UU, var_pca[:,0], 'b.-')
plt.plot(UU, var_pca[:,1], 'r.-')
plt.show()

In [ ]:
"""### VAE
cCh=16
ksize=7
fD=cCh*2*20*20
zD=16
class VAE(nn.Module):
    def __init__(self, imgCh=1): # 7: 32-26-20, 5: 32-28-24
        super().__init__()
    
        self.eC1 = nn.Conv2d(imgCh, cCh, kernel_size = ksize)
        self.eC2 = nn.Conv2d(cCh, cCh*2, kernel_size = ksize)
        self.eL1 = nn.Linear(fD, zD) 
        self.eL2 = nn.Linear(fD, zD)
    
        self.dL1 = nn.Linear(zD, fD)
        self.dC1 = nn.ConvTranspose2d(cCh*2, cCh, ksize)
        self.dC2 = nn.ConvTranspose2d(cCh, imgCh, ksize)
    
        self.actv = nn.SELU()
        self.Sigm = torch.Sigmoid()
    
    def encoder(self, x):
        x = self.actv(self.eC1(x))
        x = self.actv(self.eC2(x))
        x = x.view(-1, fD)
        mu = self.eL1(x)
        logvar = self.eL2(x)
        return mu, logvar
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(logvar/2)
        eps = torch.randn_like(std)
        return mu + std * eps
    
    def decoder(self, z):
        x = self.actv(self.dL1(z))
        x = x.view(-1, cCh*2, 20, 20)
        x = self.actv(self.dC1(x))
        x = self.Sigm(self.dC2(x))
        return x
    
    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        out = self.decoder(z)
        return out, mu, logvar"""

In [ ]:
b = 60 #inverse temp

U = np.array([0.01*i for i in range(100, 401)])
U_up, U_dn = [], []
for i, u in enumerate(U):
    if path.isfile(f'../1_QMC/train/Bethe_14_beta{b:d}/Gtau-{u:.2f}.dat')==True: U_up.append(u)
    if path.isfile(f'../1_QMC/train/Bethe_41_beta{b:d}/Gtau-{u:.2f}.dat')==True: U_dn.append(u)
U_up = np.array(U_up)
U_dn = np.array(U_dn)
spin2d = np.arange((len(U_up)+len(U_dn))*289)
spin2d = spin2d.reshape((len(U_up)+len(U_dn)), 1, 17, 17)

txt = np.zeros(289)
for i, u in enumerate(U_up):
    txt[:-1] = np.loadtxt(f'../1_QMC/train/Bethe_14_beta{b:d}/field-{u:.2f}.dat')
    spin2d[i] = txt.reshape(1, 1, 17, 17)
for i, u in enumerate(U_dn):
    txt[:-1] = np.loadtxt(f'../1_QMC/train/Bethe_41_beta{b:d}/field-{u:.2f}.dat')
    spin2d[len(U_up)+i] = txt.reshape(1, 1, 17, 17)

latent_dim = 4
beta = 4
prior_dist = dist.Normal()
q_dist = dist.Normal()


spin2d = torch.Tensor(spin2d)

In [ ]:
conv2 = nn.Conv2d(1, 16, 5, 2, 1)  # 8
bn2 = nn.BatchNorm2d(16)
conv3 = nn.Conv2d(16, 32, 4, 2, 1)  # 4
bn3 = nn.BatchNorm2d(32)
conv4 = nn.Conv2d(32, 256, 4)
bn4 = nn.BatchNorm2d(256)
conv_z = nn.Conv2d(256, 8, 1)
act = nn.SELU()

h = spin2d.view(-1, 1, 17, 17)
h = act(bn2(conv2(h)))
print(h.shape)
h.shape
h = act(bn3(conv3(h)))
h.shape
h = act(bn4(conv4(h)))
z = conv_z(h).view(spin2d.size(0), 8)
zp = z.view(spin2d.size(0), 4, 2)
zs = dist.Normal().sample(params=zp)
print(z.shape, zp.shape, zs.shape)

conv1 = nn.ConvTranspose2d(4, 256, 1, 1, 0)  # 1 x 1
bn1 = nn.BatchNorm2d(256)
conv2 = nn.ConvTranspose2d(256, 32, 4, 1, 0)  # 4
bn2 = nn.BatchNorm2d(32)
conv3 = nn.ConvTranspose2d(32, 16, 4, 2, 1)  # 8
bn3 = nn.BatchNorm2d(16)
conv_final = nn.ConvTranspose2d(16, 1, 5, 2, 1) #17
act = nn.SELU()

h = zs.view(zs.size(0), zs.size(1), 1, 1)
h = act(bn1(conv1(h)))
h = act(bn2(conv2(h)))
h = act(bn3(conv3(h)))
mu_img = conv_final(h)
mu_img.shape